<a href="https://colab.research.google.com/github/carlagarciag/PMD_HelenaMartaCarla/blob/main/caractselec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing pySpark and importing some useful packages
!pip install pyspark[sql]

from __future__ import print_function
from functools import wraps
import pyspark as spark
from pyspark import SparkConf
import time
from operator import add
import os 
from subprocess import STDOUT, check_call, check_output
from pyspark.sql import SparkSession

     |████████████████████████████████| 212.3MB 63kB/s 
     |████████████████████████████████| 204kB 19.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=cb38589cf7250fb1247f0b1ce89c0ee3cccd69b06ba2aa4907407fb1a899ae45
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
from pyspark import SparkContext
sc = SparkContext()

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [ ]:
from pyspark import SparkFiles

df = sqlContext.read.csv(SparkFiles.get("healthcare-dataset-stroke-data-preprocessed.csv"), header=True, inferSchema= True)
df.printSchema()

root
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: integer (nullable = true)
 |-- work_type: integer (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: integer (nullable = true)
 |-- stroke: integer (nullable = true)



In [ ]:
df.show()

+----+------------+-------------+------------+---------+-----------------+----+--------------+------+
| age|hypertension|heart_disease|ever_married|work_type|avg_glucose_level| bmi|smoking_status|stroke|
+----+------------+-------------+------------+---------+-----------------+----+--------------+------+
|67.0|           0|            1|           1|        2|91.88499999999999|36.6|             1|     1|
|61.0|           0|            0|           1|        3|          91.8825|28.1|             2|     1|
|80.0|           0|            1|           1|        2|           105.92|32.5|             2|     1|
|49.0|           0|            0|           1|        2|           171.23|34.4|             3|     1|
|79.0|           1|            0|           1|        3|           174.12|24.0|             2|     1|
|81.0|           0|            0|           1|        2|           186.21|29.0|             1|     1|
|74.0|           1|            1|           1|        2|            70.09|27.4|   

In [ ]:
train, test = df.randomSplit([0.7, 0.3], seed= 12345)
print("El número de pacientes para el conjunto de train es: " + str(train.count()))
print("El número de pacientes para el conjunto de test es: " + str(test.count()))

El número de pacientes para el conjunto de train es: 6717
El número de pacientes para el conjunto de test es: 2887


In [ ]:
X_train = train.select('age', 'bmi', 'avg_glucose_level', 'hypertension', 'heart_disease', 'smoking_status', 'ever_married', 'work_type')
Y_train = train.select('stroke')

X_test = train.select('age', 'bmi', 'avg_glucose_level', 'hypertension', 'heart_disease', 'smoking_status', 'ever_married', 'work_type')
Y_test = train.select('stroke')

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

# transformer
vector_assembler = VectorAssembler(inputCols=['age', 'bmi', 'avg_glucose_level', 'hypertension', 'heart_disease', 'smoking_status', 'ever_married', 'work_type'],outputCol="features")
df_temp = vector_assembler.transform(df)
df_temp.show(3)

# drop the original data features column
df = df_temp.drop('age', 'bmi', 'avg_glucose_level', 'hypertension', 'heart_disease', 'smoking_status', 'ever_married', 'work_type')
df.show(3)

from pyspark.ml.feature import StringIndexer

# estimator
l_indexer = StringIndexer(inputCol="stroke", outputCol="labelIndex")
df = l_indexer.fit(df).transform(df)
df.show(3)
# data splitting
(training,testing) = df.randomSplit([0.7,0.3])

+----+------------+-------------+------------+---------+-----------------+----+--------------+------+--------------------+
| age|hypertension|heart_disease|ever_married|work_type|avg_glucose_level| bmi|smoking_status|stroke|            features|
+----+------------+-------------+------------+---------+-----------------+----+--------------+------+--------------------+
|67.0|           0|            1|           1|        2|91.88499999999999|36.6|             1|     1|[67.0,36.6,91.884...|
|61.0|           0|            0|           1|        3|          91.8825|28.1|             2|     1|[61.0,28.1,91.882...|
|80.0|           0|            1|           1|        2|           105.92|32.5|             2|     1|[80.0,32.5,105.92...|
+----+------------+-------------+------------+---------+-----------------+----+--------------+------+--------------------+
only showing top 3 rows

+------+--------------------+
|stroke|            features|
+------+--------------------+
|     1|[67.0,36.6,91.88

ÁRBOL DE DECISIÓN


In [ ]:
%%time
from pyspark.ml.classification import DecisionTreeClassifier

# train our model using training data
dt = DecisionTreeClassifier(labelCol="labelIndex", featuresCol="features")
model = dt.fit(training)

# test our model and make predictions using testing data
predictions = model.transform(testing)
predictions.select("prediction", "labelIndex").show(5)

+----------+----------+
|prediction|labelIndex|
+----------+----------+
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
+----------+----------+
only showing top 5 rows

CPU times: user 53.9 ms, sys: 13.4 ms, total: 67.3 ms
Wall time: 5.11 s


In [ ]:
%%time
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction",metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))
print("Accuracy = %g " % accuracy)

Test Error = 0.199106 
Accuracy = 0.800894 
CPU times: user 15 ms, sys: 3.25 ms, total: 18.3 ms
Wall time: 779 ms


In [ ]:
from pyspark.mllib.evaluation import MulticlassMetrics

prediction_label = predictions.select("prediction", "labelIndex")

metricsp = MulticlassMetrics(prediction_label.rdd.map(tuple))

matrix = metricsp.confusionMatrix()

print(matrix)

DenseMatrix([[ 997.,  454.],
             [ 125., 1332.]])


especificidad = 0.6871

sensibilidad = 0.9142

REGRESIÓN LOGÍSTICA

In [ ]:
%%time
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol = 'features', labelCol = 'labelIndex', maxIter = 100)

# Fit the model
lrModel = lr.fit(training)

predictions = lrModel.transform(testing)

predictions.select("prediction", "labelIndex").show(5)

+----------+----------+
|prediction|labelIndex|
+----------+----------+
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
+----------+----------+
only showing top 5 rows

CPU times: user 61.4 ms, sys: 5.24 ms, total: 66.6 ms
Wall time: 4.51 s


In [ ]:
%%time
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction",metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))
print("Accuracy = %g " % accuracy)

Test Error = 0.206671 
Accuracy = 0.793329 
CPU times: user 7.83 ms, sys: 958 µs, total: 8.78 ms
Wall time: 458 ms


In [ ]:
prediction_label = predictions.select("prediction", "labelIndex")

metricsp = MulticlassMetrics(prediction_label.rdd.map(tuple))

matrix = metricsp.confusionMatrix()

print(matrix)

DenseMatrix([[1103.,  348.],
             [ 253., 1204.]])


especificidad = 0.7601

sensibilidad = 0.8264